In [1]:
!pip install transformers torch pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [3]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Загрузка train и test датасетов
train_df = pd.read_csv('train.csv', index_col = 0)
test_df = pd.read_csv('test.csv')

In [26]:
train_df1 = pd.read_csv('train.csv', index_col = 0)

In [6]:
# Функция для предобработки текста
def preprocess_text(text):
    if isinstance(text, str):  # Проверка типа данных
        # Приведение к нижнему регистру
        text = text.lower()
        # Удаление пунктуации и специальных символов
        text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
        # Токенизация (разбиение на слова)
        tokens = text.split()
        # Склеивание токенов обратно в текст
        cleaned_text = ' '.join(tokens)
        return cleaned_text
    else:
        return ""  # Возвращаем пустую строку для неподходящих типов данных

In [7]:
# Применение предобработки к train и test датасетам
train_df['Text'] = train_df['Text'].apply(preprocess_text)
test_df['Text'] = test_df['Text'].apply(preprocess_text)

In [8]:
# Создание экземпляра LabelEncoder для кодирования меток классов
label_encoder = LabelEncoder()

# Преобразование текстовых меток в числовой формат
train_df['Sentiment'] = label_encoder.fit_transform(train_df['Sentiment'])

In [34]:
train_df

,Text,Sentiment
0,menyrbie phil gahan chrisitv https t co ifz9fa...,3
1,advice talk to your neighbours family to excha...,4
2,coronavirus australia woolworths to give elder...,4
3,my food stock is not the only one which is emp...,4
4,me ready to go at supermarket during the covid...,0
...,...,...
41152,airline pilots offering to stock supermarket s...,3
41153,response to complaint not provided citing covi...,0
41154,you know it s getting tough when kameronwilds ...,4
41155,is it wrong that the smell of hand sanitizer i...,3


In [10]:
test_df

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending new yorkers encounter empty supermark...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,when i couldn t find hand sanitizer at fred me...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find out how you can protect yourself and love...
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,panic buying hits newyork city as anxious shop...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,toiletpaper dunnypaper coronavirus coronavirus...
...,...,...
3793,65712d27-5c41-4863-b74f-0bd66199b7df,meanwhile in a supermarket in israel people da...
3794,9fd189c5-e79c-49d7-8985-576450a4e6e3,did you panic buy a lot of non perishable item...
3795,3a06785f-6f9b-4f4d-9880-22562ad3e296,asst prof of economics cconces was on nbcphila...
3796,dd29ff09-9bc2-40f4-8201-4b6361aca760,gov need to do somethings instead of biar je r...


In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [13]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
train_encodings = tokenizer(list(train_df['Text'].values), truncation=True, padding=True, max_length=256, return_tensors='pt')
train_encodings

{'input_ids': tensor([[  101,  2273, 12541,  ...,     0,     0,     0],
        [  101,  6040,  2831,  ...,     0,     0,     0],
        [  101, 21887, 23350,  ...,     0,     0,     0],
        ...,
        [  101,  2017,  2113,  ...,     0,     0,     0],
        [  101,  2003,  2009,  ...,     0,     0,     0],
        [  101, 16985,  3775,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [15]:
test_encodings = tokenizer(list(test_df['Text'].values), truncation=True, padding=True, max_length=256, return_tensors='pt')

In [16]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = CustomDataset(train_encodings, list(train_df['Sentiment'].values))
test_dataset = CustomDataset(test_encodings)


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Убедитесь, что ваша модель перенесена на нужное устройство (CPU или GPU)

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

In [20]:
EPOCHS = 3
BATCH_SIZE = 8

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Epoch {:1d}".format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({"training_loss": "{:.3f}".format(loss.item()/len(batch))})

    avg_train_loss = total_loss / len(train_loader)
    tqdm.write(f"Average training loss: {avg_train_loss:.3f}")


Average training loss: 0.771


Average training loss: 0.446


Average training loss: 0.342


In [21]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [25]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()

        # Добавляем предсказания модели к общему списку
        predictions.extend(np.argmax(logits, axis=1).flatten())

print(predictions)  # выводит список предсказанных меток классов для вашего тестового набора данных

Predicting: 100%|██████████| 475/475 [00:24<00:00, 19.46it/s]


[0, 4, 1, 0, 3, 3, 4, 2, 0, 1, 1, 0, 0, 1, 4, 2, 4, 3, 4, 1, 0, 1, 4, 1, 0, 4, 2, 2, 0, 4, 2, 1, 0, 0, 4, 0, 2, 3, 2, 1, 4, 2, 3, 4, 4, 1, 1, 2, 0, 0, 0, 3, 4, 1, 0, 2, 2, 1, 4, 3, 0, 0, 1, 0, 4, 4, 1, 1, 1, 1, 0, 1, 4, 2, 3, 0, 2, 2, 4, 4, 0, 0, 1, 2, 4, 2, 2, 3, 2, 2, 2, 2, 4, 0, 2, 1, 3, 1, 0, 3, 4, 3, 4, 4, 3, 1, 4, 2, 2, 0, 2, 4, 1, 2, 4, 2, 0, 1, 2, 3, 4, 4, 4, 2, 0, 0, 1, 1, 2, 0, 0, 4, 0, 0, 0, 1, 1, 2, 1, 3, 1, 1, 2, 2, 4, 4, 2, 4, 2, 4, 2, 1, 2, 2, 3, 2, 2, 2, 4, 0, 4, 4, 1, 3, 3, 4, 2, 4, 1, 1, 2, 1, 3, 4, 1, 4, 3, 4, 2, 2, 3, 1, 3, 4, 2, 1, 3, 4, 4, 2, 1, 1, 1, 1, 4, 4, 3, 4, 1, 2, 4, 4, 4, 1, 4, 0, 2, 1, 3, 4, 0, 2, 3, 4, 1, 2, 0, 3, 3, 0, 3, 4, 2, 2, 4, 1, 4, 4, 4, 0, 0, 4, 4, 3, 3, 4, 1, 3, 4, 4, 1, 4, 2, 2, 0, 0, 4, 2, 3, 2, 3, 3, 4, 1, 4, 4, 4, 4, 1, 0, 2, 1, 0, 4, 2, 2, 4, 0, 4, 2, 3, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 3, 4, 4, 0, 4, 2, 3, 0, 0, 2, 2, 2, 2, 2, 1, 4, 4, 4, 3, 2, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2, 4, 2, 2, 2, 2, 4, 0, 1, 4, 3, 4, 2, 2, 2, 4, 1, 4, 3, 4, 3, 1, 2, 

In [36]:
test_df['pred'] = predictions
test_df.head()

,id,Text,pred
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending new yorkers encounter empty supermark...,0
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,when i couldn t find hand sanitizer at fred me...,4
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find out how you can protect yourself and love...,1
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,panic buying hits newyork city as anxious shop...,0
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,toiletpaper dunnypaper coronavirus coronavirus...,3
...,...,...,...
3793,65712d27-5c41-4863-b74f-0bd66199b7df,meanwhile in a supermarket in israel people da...,4
3794,9fd189c5-e79c-49d7-8985-576450a4e6e3,did you panic buy a lot of non perishable item...,2
3795,3a06785f-6f9b-4f4d-9880-22562ad3e296,asst prof of economics cconces was on nbcphila...,3
3796,dd29ff09-9bc2-40f4-8201-4b6361aca760,gov need to do somethings instead of biar je r...,0


In [35]:
def pred_conversion(text):

    if text == 0:
        return 'Extremely Negative'
    elif text == 1:
        return 'Extremely Positive'
    elif text == 2:
        return 'Negative'
    elif text == 3:
        return 'Neutral'
    else:
        return 'Positive'

In [37]:
test_df['Sentiment'] = test_df['pred'].apply(lambda x: pred_conversion(x))
test_df

,id,Text,pred,Sentiment
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending new yorkers encounter empty supermark...,0,Extremely Negative
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,when i couldn t find hand sanitizer at fred me...,4,Positive
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find out how you can protect yourself and love...,1,Extremely Positive
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,panic buying hits newyork city as anxious shop...,0,Extremely Negative
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,toiletpaper dunnypaper coronavirus coronavirus...,3,Neutral
...,...,...,...,...
3793,65712d27-5c41-4863-b74f-0bd66199b7df,meanwhile in a supermarket in israel people da...,4,Positive
3794,9fd189c5-e79c-49d7-8985-576450a4e6e3,did you panic buy a lot of non perishable item...,2,Negative
3795,3a06785f-6f9b-4f4d-9880-22562ad3e296,asst prof of economics cconces was on nbcphila...,3,Neutral
3796,dd29ff09-9bc2-40f4-8201-4b6361aca760,gov need to do somethings instead of biar je r...,0,Extremely Negative


In [39]:
sub = test_df[['id', 'Sentiment']]
sub.to_csv('sub.csv',index = False )